# 人生计算器

In [11]:
import numpy as np
import altair as alt
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [14]:
def life_calculator(years, initial_savings, salary_func, investment_func, spending_func):
    """
    计算并返回未来几年的每年净收入，考虑资金累积。
    - years: 预测年数
    - initial_savings: 初始存款金额
    - salary_func: 接收年份，返回每年的税后收入数组
    - investment_func: 接收年份和上年末余额，返回每年的投资收入数组
    - spending_func: 接收年份，返回每年的花销数组
    """
    years_array = np.arange(1, years + 1)
    savings = np.zeros(years)  # 初始化一个存储每年末余额的数组
    savings[0] = initial_savings  # 设定初始存款
    
    for i in range(years):
        year = years_array[i]
        current_salary = salary_func(year)
        current_spending = spending_func(year)
        
        if i == 0:
            previous_savings = initial_savings
        else:
            previous_savings = savings[i - 1]
        
        # 计算投资收入
        current_investment_income = investment_func(year, previous_savings)
        
        # 更新余额
        savings[i] = previous_savings + current_salary + current_investment_income - current_spending
    
    # 净收入是每年的增长量
    net_income = savings - np.roll(savings, 1)
    net_income[0] = savings[0] - initial_savings  # 第一年的净收入特殊处理
    
    return years_array, net_income, savings

In [15]:
# 示例函数
def salary_example(years):
    # 假设起始年薪为税后120,000美元，并每年增长5%
    return 120000 * (1.05 ** (years - 1))



def spending_example(years):
    # 假设每年的花销为20,000美元，并每年增长3%
    return 20000 * (1.03 ** (years - 1))

In [18]:
@interact(investment_return_per_year=(0.0,1.0,0.01))
def interactive_plot(investment_return_per_year):

  def investment_func(year, previous_savings):
      # 假设每年的投资回报率为7%
      return previous_savings * investment_return_per_year

  years, net_income, total_savings = life_calculator(30, 80000, salary_example, investment_func, spending_example)
  income_target = 365000
  savings_target = 10000000


  data = pd.DataFrame({
      'Year': years,
      'Net Income': net_income,
      'Total Savings': total_savings
    })
  def find_first_year(data, column, threshold):
      """找到数据首次达到某一阈值的年份"""
      years = data['Year'][data[column] >= threshold]
      if len(years) > 0:
          return years.iloc[0]
      else:
          return None  # 如果没有达到目标，则返回 None
  income_target_year = find_first_year(data, 'Net Income', income_target)
  savings_target_year = find_first_year(data, 'Total Savings', savings_target)

  base = alt.Chart(data).encode(x='Year:O')

  # 网络收入图
  net_income_chart = base.mark_line(color='blue', point=True).encode(
      y='Net Income:Q',
      tooltip=['Year', 'Net Income']
  ).properties(
      title="Net Income Over Years"
  )
  # 总储蓄图
  savings_chart = base.mark_line(color='green', point=True).encode(
      y='Total Savings:Q',
      tooltip=['Year', 'Total Savings']
  ).properties(
      title="Total Savings Over Years"
  )
  # 如果有达到目标的年份，则添加标记线
  if income_target_year:
      income_target_line = alt.Chart(pd.DataFrame({'Year': [income_target_year]})).mark_rule(color='red', strokeDash=[6,2]).encode(
          x='Year:O'
      )
      net_income_line = alt.Chart(pd.DataFrame({'y': [income_target]})).mark_rule(color='red', strokeDash=[6,2]).encode(
          y='y:Q'
      )
      net_income_chart += income_target_line + net_income_line
  if savings_target_year:
      savings_target_line = alt.Chart(pd.DataFrame({'Year': [savings_target_year]})).mark_rule(color='red', strokeDash=[6,2]).encode(
          x='Year:O'
      )
      # 添加总储蓄达到 10,000,000 美元的水平线
      savings_line = alt.Chart(pd.DataFrame({'y': [savings_target]})).mark_rule(color='red', strokeDash=[6,2]).encode(
          y='y:Q'
      )
      savings_chart += savings_target_line + savings_line


  return net_income_chart | savings_chart  # 并排显示两个图表

interactive(children=(FloatSlider(value=0.5, description='investment_return_per_year', max=1.0, step=0.01), Ou…